In [ ]:
!pip install pymupdf
#!pip install PyPDF2
#!pip install fpdf
!pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 43.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40702 sha256=c9ab5d6a364c5fa78061385a9d1e3f580e468f09d8d13ca838232a172b630e39
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 6.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import fitz  # PyMuPDF
from jellyfish import jaro_winkler_similarity
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.platypus.flowables import KeepTogether

# Membaca dataset dari file Excel
dataset_path = '/content/1000dataset.xlsx'
df = pd.read_excel(dataset_path)

data_frame = df['KATA']

# Menyimpan kata-kata dalam list
word_list = data_frame.tolist()
word_list_with_dot = (data_frame + '.').tolist()
word_list_with_comma = (data_frame + ',').tolist()

word_list = word_list + word_list_with_dot + word_list_with_comma
total_dataset = len(word_list)

# Menampilkan dataset
print(word_list)

# Menampilkan total dataset
print(f"Total dataset yang tersedia: {total_dataset}")

# Path to the uploaded PDF file
pdf_path = '/content/typo_dokumen2.pdf'

def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    document = fitz.open(pdf_path)
    text = []
    for page_num in range(document.page_count):
        page = document[page_num]
        text.append(page.get_text("text"))
    return text

# Extract and print the text from the PDF
document_text = extract_text_from_pdf(pdf_path)
for page_text in document_text:
    print(page_text)

def detect_and_correct_typos(text, word_list, threshold=0.94):
    corrected_text = []
    typos = []
    corrected_words = []
    for line in text:
        words = line.split()
        corrected_line = []
        for word in words:
            best_match = word
            highest_similarity = 0
            for dict_word in word_list:
                similarity = jaro_winkler_similarity(word, dict_word)
                if similarity > highest_similarity:
                    highest_similarity = similarity
                    best_match = dict_word
            if highest_similarity < threshold and highest_similarity > 0:
                typos.append((word, best_match, highest_similarity))
            if best_match != word:
                corrected_words.append((word, best_match, highest_similarity))
            corrected_line.append((word, best_match, highest_similarity))
        corrected_text.append(corrected_line)
    return corrected_text, typos, corrected_words

corrected_text, typos, corrected_words = detect_and_correct_typos(document_text, word_list)

# Display the identified typos with their most similar word from the dataset and similarity score
print("\nDetected Typos:")
for typo in typos:
    print(f"Word: {typo[0]}, Most Similar Word: {typo[1]}, Similarity: {typo[2]:.2f}")

# Display the corrected words with similarity score
print("\nCorrected Words:")
for original, corrected, similarity in corrected_words:
    print(f"Original: {original}, Corrected: {corrected}, Similarity: {similarity:.2f}")

# Save the corrected text to a new PDF file with highlighted corrections
def save_text_to_pdf(corrected_text, output_path):
    doc_title = output_path.split('/')[-1]  # Extract the filename from the output path
    doc = SimpleDocTemplate(output_path, pagesize=A4, title=doc_title)
    styles = getSampleStyleSheet()
    normal_style = styles['Normal']
    corrected_style = ParagraphStyle(
        name='Corrected',
        parent=normal_style,
        textColor=colors.red,
    )

    story = []
    for line in corrected_text:
        line_text = ""
        for word, corrected, similarity in line:
            if corrected != word and similarity >= 0.94:
                line_text += f'<font color="red">{word}</font> '
            else:
                line_text += f'{word} '
        story.append(KeepTogether([Paragraph(line_text.strip(), normal_style)]))
        story.append(Spacer(1, 12))

    doc.build(story)

output_pdf_path = '/content/corrected_typo_dokumen2.pdf'
save_text_to_pdf(corrected_text, output_pdf_path)

print(f"Corrected PDF saved to {output_pdf_path}")


['abang', 'adik', 'adonan', 'air', 'air conditioner', 'teman', 'akuarium', 'Al-Quran', 'alamat', 'alat', 'alfabet', 'alis', 'Allah', 'alpukat', 'ambulan', 'amplop', 'anak', 'anggur', 'angin', 'angin', 'angklung', 'angkot', 'anjing', 'anjing laut', 'anting', 'apartemen', 'apel', 'api', 'apotek', 'asap', 'asbak', 'atap', 'awan', 'ayah', 'ayam goreng', 'ayunan', 'babi', 'badak', 'badan', 'badut', 'bajaj', 'baju', 'bak mandi', 'balok', 'balon', 'ban', 'bandara', 'bando', 'bangau', 'banjir', 'bank', 'bantal', 'banteng', 'bapak', 'bapak guru', 'batang', 'baterai', 'batik', 'batu', 'bayangan', 'bayi', 'bebek', 'becak', 'bedak', 'bel', 'belalai', 'belalang', 'belimbing', 'belut', 'benang', 'bendera', 'bengkel', 'beras', 'beruang', 'besi', 'bibir', 'biji', 'binatang', 'bintang', 'bintang laut', 'biola', 'bioskop', 'bis', 'biskuit', 'boks', 'bola', 'bolu', 'boneka', 'botol', 'brokoli', 'bros', 'buah', 'buaya', 'bubur', 'buku cerita', 'buku gambar', 'buku tulis', 'bulan', 'buncis', 'bunda', 'bung